### Transforming data

- turn the data into tensors
- turn into dataset (torch.utils.data.Dataset) then into data loader (torch.utils.data.DataLoader)

@refer: https://pytorch.org/vision/main/generated/torchvision.datasets.ImageFolder.html#torchvision.datasets.ImageFolder

In [1]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms


Transforming data using torchvision.transforms

In [2]:
data_transform = transforms.Compose([
    # Resize the images to 64x64
    transforms.Resize(size=(64,64)),
    # flip the images on the horizontal axis
    transforms.RandomHorizontalFlip(p=0.5),
    # Converts a PIL Image or numpy.ndarray (H x W x C) in the range [0, 255] to a torch.FloatTensor of shape (C x H x W) in the range [0.0, 1.0]
    transforms.ToTensor()
])